In [153]:
from urllib import request
from urllib.request import urlopen
from urllib.parse import urlencode, quote, unquote
import requests
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm
import time
from selenium import webdriver

In [119]:
def create_driver(state):
    driver = webdriver.Chrome('C:/Users/tianm/chromedriver_win32/chromedriver')
    if state == "onGoing":
        driver.get("https://petitions.assembly.go.kr/status/onGoing")
    elif state == "closed":
        driver.get("https://petitions.assembly.go.kr/closed/inadequate")
        driver.find_element_by_id('beginDate').send_keys("01")
        driver.find_element_by_css_selector('#contentsbody > div > div > div.board_info.datesearch > div > fieldset > div > input.btn_search').click()
    time.sleep(3)
    return driver

In [120]:
def get_page_num(driver):
    petition_num = driver.find_element_by_css_selector('#contentsbody > div > div > div.petitionListWrap > div.petitionListDiv > div.conditionDiv > div > span').text
    page_num = int(petition_num)//8 + 1
    return page_num

In [128]:
def get_links(driver, page_num):
    links = []
    for j in range(0, page_num):  
        if state == 'onGoing':
            last = 6
        elif state == 'closed':
            last = 11
        click_to_last_page = f'#contentsbody > div > div > div.petitionListWrap > div.pagination-area > ul > li:nth-of-type({last}) > a'
        
        if j == 0:
            driver.find_element_by_css_selector(click_to_last_page).click()
            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')
            link = soup.select('#contentsbody > div > div > div.petitionListWrap > div.petitionListDiv > div.ListDiv > ul > li > div > a')
            links.extend(link)
        else:
            driver.find_element_by_css_selector('#contentsbody > div > div > div.petitionListWrap > div.pagination-area > ul > li:nth-child(2) > a').click()
            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')
            link = soup.select('#contentsbody > div > div > div.petitionListWrap > div.petitionListDiv > div.ListDiv > ul > li > div > a')
            links.extend(link)
        time.sleep(1)   
    driver.close()
    return links

In [129]:
def cleaning_links(links):
    clean_links = []
    for link in links:
        clean_links.append('https://petitions.assembly.go.kr' + link['href'])
    return clean_links

In [123]:
state = 'closed'
driver = create_driver(state)
page_num = get_page_num(driver)
links = get_links(driver, page_num)
clean_links_1 = cleaning_links(links)

In [130]:
state = 'onGoing'
driver = create_driver(state)
page_num = get_page_num(driver)
links = get_links(driver, page_num)
clean_links_2 = cleaning_links(links)

In [132]:
clean_links = clean_links_1 + clean_links_2
len(clean_links)

220

In [167]:
petitions = []

In [169]:
for i in tqdm(range(0, len(clean_links))):
    link = clean_links[i]
    driver = webdriver.Chrome('C:/Users/tianm/chromedriver_win32/chromedriver')
    driver.get(link)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
        
    if link.find("onGoing") >= 0:
        state = "청원진행중"
    elif link.find("closed") >= 0:
        state = "청원종료"
    title = soup.select_one('#contentsbody > div > div > div.viewTopDiv > h4').text
    categ = soup.select_one('#contentsbody > div > div > div.petitionViewWrap > div.petitionfield > ul.firstline > li > div').text
    date = soup.select_one('#contentsbody > div > div > div.petitionViewWrap > div.petitionfield > ul.secondline > li:nth-of-type(1) > div').text.rstrip('\n(청원서 공개 이후 30일 이내)')
    count = soup.select_one('#contentsbody > div > div > div.petitionViewWrap > div.petitionfield > ul.secondline > li:nth-of-type(2) > div > div > ul > li:nth-of-type(1)').text.lstrip('동의자수\n')
    text1 = soup.select_one('#contentsbody > div > div > div.petitionViewWrap > div.petitionTb > dl:nth-of-type(2) > dd').text
    text2 = soup.select_one('#contentsbody > div > div > div.petitionViewWrap > div.petitionTb > dl:nth-of-type(3) > dd').text
    
    petitions.append([state, title, count, categ, date, text1, text2])
    
    driver.close()
    time.sleep(1)

100%|████████████████████████████████████████████████████████████████████████████████| 220/220 [37:21<00:00, 10.19s/it]


In [210]:
petitions = pd.DataFrame(petitions) # date는 나누고 text는 합쳐야 함. 

state = petitions[0]
title  = petitions[1].str.rstrip("\n    ")
count = petitions[2].str.rstrip("명\n    ") 
categ = petitions[3]
start = petitions[4].str.lstrip("\n            ").str.split("~", expand = True)[0]
end = petitions[4].str.lstrip("\n            ").str.split("~", expand = True)[1]
text = petitions[5].map(str) + " " + petitions[6]

In [211]:
d = {'state': state, "title":title, "count": count, "categ": categ, "start":start, "end": end, "text":text }
df = pd.DataFrame(data=d)
df

,state,title,count,categ,start,end,text
0,청원종료,국가보안법 개정에 관한 청원,"1,761",수사/법무/사법제도,2021-06-16,2021-07-18,최근 국가보안법 폐지 청원이 올라온것을 보았으나 자유대한민국이 월남의 전철을 밟아가...
1,청원종료,인공지능 윤리 및 고인의 AI구현 법안 제정에 관한 청원,223,과학기술/정보통신,2021-06-15,2021-07-17,동의 없는 AI구현에 대한 제재를 가할 수 있도록 규정을 만들고 그 외의 AI구현을...
2,청원종료,포괄적 차별금지법 법안 제정에 관한 동의 및 일부 내용 수정 요청에 관한 청원,"14,025",인권/성평등/노동,2021-06-15,2021-07-17,"UN 사회권위원회에서는 12년, 국가인권위원회에서는 15년째 포괄적 차별금지법을 제..."
3,청원종료,국민연금 수급 개시 연령과 연계한 정년연장에 관한 청원,"19,807",인권/성평등/노동,2021-06-14,2021-07-16,노동자의 정년의 기간을 국민연금 수급 개시 연령과 연계하여 “고용상 연령차별금지 및...
4,청원종료,하수도법 일부개정법률안에 관한 청원,736,재난/안전/환경,2021-06-11,2021-07-1,"하수도법에의한 특정공산품(주방용오물분쇄기)은 한미 FTA 무역협정 공산품이며, 특정..."
...,...,...,...,...,...,...,...
215,청원진행중,심리상담서비스법 제정에 관한 청원,226,복지/보훈,2021-07-07,2021-08-08,"최근 한국심리학회(상담심리학회, 임상심리학회 포함 산하 15개 분과학회)와 한국상담..."
216,청원진행중,온라인 플랫폼 중개거래의 공정화에 관한 법률안 제정에 관한 청원,169,소비자/공정거래,2021-06-25,2021-07-27,"출판계와 플랫폼의 갑질을 막아주세요 한국의 출판계의 미래와 웹소설, 웹툰 작가들을 ..."
217,청원진행중,대체휴일에 관한 청원,161,행정/지방자치,2021-07-14,2021-08-1,"토, 일요일의 공휴일에 대한 대체휴일을 일괄적으로 다음 월요일로 정하는 것에 대한 ..."
218,청원진행중,셧다운제 폐지를 위한 청소년 보호법 개정에 관한 청원,152,저출산/고령화/아동/청소년/가족,2021-07-07,2021-08-08,순기능은 제대로 하지 못한채 역효과만 일으키고 있는 셧다운제(청소년 보호법 제26조...


In [213]:
df.to_excel("D:\Summer 2021\국회.xlsx", index = False)